# Simple FEnicsx Tutorial

This tutorial demonstrates using FEnicsx to solve the Helmholtz Equation.

This is only a slight modification of the tutorial found at https://jsdokken.com/fenics22-tutorial/helmholtz.html - please explore this site to view the original and for other in-depth tutorials.

### References
Hans Petter Langtangen and Anders Logg. Solving PDEs in Python: The FEniCS Tutorial I. Springer International Publishing, Cham, 2016. ISBN 978-3-319-52462-7. doi:10.1007/978-3-319-52462-7

## Problem Setup

In [1]:
from mpi4py import MPI

import numpy as np

import dolfinx.fem.petsc
import ufl

This example is designed to be executed with complex-valued degrees of freedom. To be able to solve this problem, we use the complex build of PETSc.

In [5]:
import sys

from petsc4py import PETSc

#if not np.issubdtype(PETSc.ScalarType, np.complexfloating):
#    print("This tutorial requires complex number support")
#    sys.exit(0)
#else:
#    print(f"Using {PETSc.ScalarType}.")

## Defining Model Parameters

In [7]:
# wavenumber in free space (air)
k0 = 10 * np.pi

# Corresponding wavelength
lmbda = 2 * np.pi / k0

# Polynomial degree
degree = 6

# Mesh order
mesh_order = 2

## Interfacing with GMSH

We will use Gmsh to generate the computational domain (mesh) for this example. As long as Gmsh has been installed (including its Python API), DOLFINx supports direct input of Gmsh models (generated on one process). DOLFINx will then in turn distribute the mesh over all processes in the communicator passed to dolfinx.io.gmshio.model_to_mesh.

The function generate_mesh creates a Gmsh model on rank 0 of MPI.COMM_WORLD.

In [8]:
from dolfinx.io import gmshio
from mesh_generation import generate_mesh

# MPI communicator
comm = MPI.COMM_WORLD

file_name = "domain.msh"
generate_mesh(file_name, lmbda, order=mesh_order)
mesh, cell_tags, _ = gmshio.read_from_msh(file_name, comm, rank=0, gdim=2)

ModuleNotFoundError: No module named 'mesh_generation'

In [ ]:
## Material Parameters

Material parameters
In this problem, the wave number in the different parts of the domain depends on cell markers, inputted through cell_tags.
We use the fact that a discontinuous Lagrange space of order 0 (cell-wise constants) has a one-to-one mapping with the cells local to the process.

In [ ]:
W = dolfinx.fem.functionspace(mesh, ("DG", 0))
k = dolfinx.fem.Function(W)
k.x.array[:] = k0
k.x.array[cell_tags.find(1)] = 3 * k0

In [ ]:
import matplotlib.pyplot as plt
import pyvista

from dolfinx.plot import vtk_mesh

pyvista.start_xvfb()
pyvista.set_plot_theme("paraview")
sargs = dict(
    title_font_size=25,
    label_font_size=20,
    fmt="%.2e",
    color="black",
    position_x=0.1,
    position_y=0.8,
    width=0.8,
    height=0.1,
)


def export_function(grid, name, show_mesh=False, tessellate=False):
    grid.set_active_scalars(name)
    plotter = pyvista.Plotter(window_size=(700, 700))
    t_grid = grid.tessellate() if tessellate else grid
    plotter.add_mesh(t_grid, show_edges=False, scalar_bar_args=sargs)
    if show_mesh:
        V = dolfinx.fem.functionspace(mesh, ("Lagrange", 1))
        grid_mesh = pyvista.UnstructuredGrid(*vtk_mesh(V))
        plotter.add_mesh(grid_mesh, style="wireframe", line_width=0.1, color="k")
        plotter.view_xy()
    plotter.view_xy()
    plotter.camera.zoom(1.3)
    plotter.export_html(f"./{name}.html")

In [ ]:
grid = pyvista.UnstructuredGrid(*vtk_mesh(mesh))
grid.cell_data["wavenumber"] = k.x.array.real

In [ ]:
export_function(grid, "wavenumber", show_mesh=True, tessellate=True)

In [ ]:
%%html
<iframe src='./wavenumber.html' height="700px" width="700px"></iframe>  <!--  # noqa, -->